[LEGO Catalog Database Download](https://rebrickable.com/downloads/)

![Rebrickable ERD](https://rebrickable.com/static/img/diagrams/downloads_schema_v3.png)

In [47]:
import pandas as pd
import glob

def read_all_csv_files_in_current_folder():
    csv_files = [
        'colors.csv',
        'elements.csv',
        'inventories.csv',
        'inventory_minifigs.csv',
        'inventory_parts.csv',
        'inventory_sets.csv',
        'minifigs.csv',
        'parts.csv',
        'part_categories.csv',
        'part_relationships.csv',
        'sets.csv',
        'themes.csv'
    ]
    
    dataframes = {}
    
    for csv_file in csv_files:
        df_name = csv_file[:-4] + '_df'
        df = pd.read_csv(csv_file)
        dataframes[df_name] = df
    
    return dataframes

# Call the function to read all CSV files into DataFrames
result = read_all_csv_files_in_current_folder()

# Access individual DataFrames using the modified names
colors_df = result['colors_df']
elements_df = result['elements_df']
inventories_df = result['inventories_df']
inventory_minifigs_df = result['inventory_minifigs_df']
inventory_parts_df = result['inventory_parts_df']
inventory_sets_df = result['inventory_sets_df']
minifigs_df = result['minifigs_df']
parts_df = result['parts_df']
part_categories_df = result['part_categories_df']
part_relationships_df = result['part_relationships_df']
sets_df = result['sets_df']
themes_df = result['themes_df']

# Now you can use the individual DataFrames as needed in your code


In [48]:
colors_df.head()

,id,name,rgb,is_trans
0,-1,[Unknown],0033B2,f
1,0,Black,05131D,f
2,1,Blue,0055BF,f
3,2,Green,237841,f
4,3,Dark Turquoise,008F9B,f
